### Milestone 4: Deep learning, due Wednesday, April 26, 2017

For this milestone you will (finally) use deep learning to predict movie genres. You will train one small network from scratch on the posters only, and compare this one to a pre-trained network that you finetune. [Here](https://keras.io/getting-started/faq/#how-can-i-use-pre-trained-models-in-keras) is a description of how to use pretrained models in Keras.

You can try different architectures, initializations, parameter settings, optimization methods, etc. Be adventurous and explore deep learning! It can be fun to combine the features learned by the deep learning model with a SVM, or incorporate meta data into your deep learning model. 

**Note:** Be mindful of the longer training times for deep models. Not only for training time, but also for the parameter tuning efforts. You need time to develop a feel for the different parameters and which settings work, which normalization you want to use, which model architecture you choose, etc. 

It is great that we have GPU's via AWS to speed up the actual computation time, but you need to be mindful of your AWS credits. The GPU instances are not cheap and can accumulate costs rather quickly. Think about your model first and do some quick dry runs with a larger learning rate or large batch size on your local machine. 

The notebook to submit this week should at least include:

- Complete description of the deep network you trained from scratch, including parameter settings, performance, features learned, etc. 
- Complete description of the pre-trained network that you fine tuned, including parameter settings, performance, features learned, etc. 
- Discussion of the results, how much improvement you gained with fine tuning, etc. 
- Discussion of at least one additional exploratory idea you pursued


# Training and Testing data

The training set consisted of 7814 RGB movie posters of size 500 x741 and the testing set consisted of 3700 futher similar images. 

Before passing the images into the model, the images were rescaled to 100x 148 and the pixel values were normalised by dividing by 255. 

# Deep network trained from scratch

We built a Convolutional neural network from scratch. CNNs are superior to MLPs in image processing as pixels are correlated and the convolutional layers preserve pixel neighborhood structure.  

Our CNN model is built with a stack of 3 convolutional + max pool layers with a ReLU activation. We then include a single fully-connected layer followed by an output layer which has 7 units with sigmoid activation, giving a 7 dimensional output vector (0 or 1 for each possible genre).  To train the model we chose to use categorical crossentropy loss, a batch size of 16 and the optimizer “Adam,” an algorithm for first-order gradient-based optimization of stochastic objective functions.

Next, we used grid search, a model hyperparameter optimization technique. The grid search then constructs and evaluates one model for each combination of parameters. We then apply 4 fold cross validation (cv=4)  to evaluate each individual model. The outcome of the grid search including the the best score observed during the optimization procedure and the the combination of parameters that achieved the best results.

After running grid search (and crashing multiple times on our CNN), we found reasonable set of params that we do better. A couple changes and their justifications:

- We learned that more neurons/feature detectors with dropout is more effective than just less neurons.
- Adam optimizer and rmsprop do not make a difference
- Categorical crossentropy seem to slightly outperform binary crossentropy when it comes to multi-label predictions
- The number of epochs past 20 doesn't improve performance too much. We were surprised at this because 20 epochs seem low, but it seems it's good enough given our small size of data.

# Fine tuned pretrained network 

To leverage results and computational power from prior work, we used a VVGnet model with pretrained weights from Imagenet. The network was truncated before the fully connected output layer, preserving only the convolution/max pooling layers. 

First, we passed the testing and training data through the network to extract out the output feature vector. 

This was fed as the input into a a MLP with two fully connected layers and used to predict the genre labels. 

For our second approach we froze all but the last convolutional block of the VVGnet model and attached our own fully connected output block consisting of two fully connected layers with ReLU activation each followed by 50% dropout and a final layer with 7 units producing a 7 dimensional binary vector. 

In the second model, the weights of the last convolution block of VVGnet are also being updated by backprop as opposed to the first model in which VVGnet was simply used for feature extraction 

The hybrid model was trained for 50 epochs with 'adam' as the optimizer and a batch size of 50 and evaluated using the 'binary cross entropy' loss. 

# Discussion of the results

We measured the performace of the models on the multi-label task using the following error measures:  Hamming Loss, Percent exact match, and Percent at least one label matches. 

The results from the four models is shown in the table below:

| model | hamming loss | percent exact match | percent at least one match |
|-----------|----|----|--------|
| Deep network from scratch        | 0.154 |  43.8%  |    94.8%    |
| Deep network from scratch  + parameter tuning         | 0.13  | 49.7%  | 95.7%     |
| Pretrained model + MLP       |   |  |     |
| Pretrained model + fine tuning of last convolution block  + MLP    |   |    |       | |

Parameter tuning of the basic CNN model resulted in the hamming loss decreasing from 0.15 to 0.13, percent exact increasing from 0.44 to 0.50, and percent at least one increasing slightly. We did not expect to achieve significalty greater improvements through further fine tuning of this model so did not pursue this model further. 

Extracting features from the pretrained VVGnet model and feeding into a custom MLP, resulted in accuracies of 



The hamming loss has improved by about 2% (as the score went down 2%), suggesting that the label-based accuracy for the multi-label case has improved as the fraction of wrong labels to the total number of labels has improved. The percent exact has improved by about 6%, and the percent at least one has improved by 1%. 


### Problems Encountered 

During the building and training of these models we encountered several problems and had to devise innovative solutions:
- the original poster images were too large and caused memor issues when run on AWS. The resolution had to be decreased to enable the models to be trained. 
- posters had to be loaded from external storage (we chose to use github) and took several minutes. We therefore limited our training data set to 7814 images to void having to wait long periods to load data before training, and enable us to iterate faster with our models.  

# Additional exploratory idea

In our additional exploration, we read the popular paper 'A Neural Algorithm of Artistic Style' (Gatys et al., 2015 - https://arxiv.org/abs/1508.06576) that demonstrates how to use neural networks to transfer artistic style from one image onto another. This is the underlying technology behind the popular Prisma app.

We adapted our code to transfer styles between different genres of posters. The full details of the model are given in the notebook with the code but the key features of the model are outlined below: